# NYC Taxis Fare Prediction Analysis Report

Authors: Han Wang, Jam Lin, Jiayi Li, Yibin Long

In [ ]:
import pandas as pd
import altair as alt
import numpy as np


data_set_link = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"

# Use only a smaller, random subset (30,000 rows) of the data
df = pd.read_parquet(data_set_link).sample(30000, random_state=123)
df.to_csv('data/yellow_tripdata_2024-01.csv', index=False)


# Preface

This report was developed as a deliverable for the term project in DSCI 522 (Data Science Workflows), a course in the Master of Data Science program at the University of British Columbia.

The overall objective of this project was to automate a typical data science workflow. This report summarizes the results of a series of automated Python scripts that handle tasks such as data retrieval, data cleaning, exploratory data analysis (EDA), creation of a predictive machine learning model, and interpretation of the results. The report provides a detailed explanation of each step, applying it to the specific context of the dataset in question. It assumes the reader has a basic understanding of machine learning terminology and concepts.

# Summary

This report presents a linear regression model developed to predict NYC taxi fare amount. Using data from 30,000 taxi trips in January 2024, we build a linear regression model using trip distance feature, with each additional mile increasing the fare by approximately $3.54. The model was evaluated using a test dataset, where predicted fare amounts were compared to actual fares. While the model performed well overall, some outliers were identified, suggesting the need for further data cleaning and additional features to improve accuracy. Future steps include incorporating more features, experimenting with other regression models like KNN and Lasso regression, and addressing hyperparameters for better generalizability and performance.

# Introduction/Background

Taking a taxi in New York City can be overwhelming for first-time visitors, especially for tourists unfamiliar with the city's layout and fare system. With over 200,000 taxi trips taking place daily, yellow cabs remain an essential mode of transportation for both locals and visitors. However, the lack of transparency often leads to concerns among tourists about overcharging or being taken on unnecessarily long routes.

To address these concerns, we use data from 30,000 Yellow Taxi trips recorded in January 2024, provided by the NYC Taxi and Limousine Commission (TLC). Our analysis examines how to predict taxi fare amounts, offering valuable insights to help tourists better understand what they should expect to pay for a taxi ride in NYC. By leveraging data-driven analysis, we aim to provide a clearer and more predictable fare structure, enabling tourists to make more informed decisions during their travels in the city.

# Analysis Question:
How do we best predict NYC yellow taxi fare amounts?

# Dataset

The data set used in this project is the TLC Trip Record Data, which includes taxi and for-hire vehicle trip records collected by technology providers authorized under the Taxicab & Livery Passenger Enhancement Programs (TPEP/LPEP). This data is sourced from the NYC Taxi and Limousine Commission (TLC) and can be accessed from the NYC Taxi and Limousine Commission's [website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).
Yellow and green taxi trip records contain fields capturing pick-up and drop-off dates and times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. Similarly, For-Hire Vehicle (FHV) trip records include fields such as dispatching base license number, pick-up date and time, and taxi zone location ID.
It is important to note that the TLC did not create this data and makes no representations regarding its accuracy. FHV trip records are based on submissions from dispatching bases and may not represent the total volume of trips. Additionally, the TLC reviews these records and enforces necessary actions to improve their accuracy and completeness.

# Methods and Results

## Methods

In order to address our research question, we will begin by selecting the appropriate features from the dataset through exploratory data analysis (EDA) and by consulting the data dictionary to better understand the instances within the dataset (dataset information is linked in the references). Since this is a linear regression modeling problem, we will use the LinearRegression model as our model of choice. Linear regression is a fundamental and widely used method for predictive modeling.
The dataset used in this study includes daily fare amounts and trip distance data from January 2024, covering 30,000 observations. The data will be split into training and test sets, with 70% allocated for training and 30% for testing.

The analysis will be conducted using the Python programming language [Van Rossum and Drake, 2009], with the following Python packages: numpy [Harris et al., 2020], pandas [McKinney, 2010; VanderPlas, 2018], scikit-learn [Pedregosa et al., 2011], and altair [VanderPlas et al., 2018].


## EDA

The exploratory data analysis (EDA) reveals key insights about the dataset, particularly the target variable, fare_amount. Most fares are under $50, with a cluster around $65, representing longer trips, while negative fares were excluded. The mean fare is $18.59, and the median is $12.8, indicating a right-skewed distribution. Null values in some columns have minimal impact unless used in analyses. A strong correlation between trip_distance and fare_amount supports using trip_distance as a predictor. Data validation confirmed completeness in critical columns, with missing data under 5%. (Note: Summary statistics and correlation plots can be found in the charts folder.)

## Modeling

For the modeling process, a Linear Regression model was built and tested. Error metrics were calculated to assess its performance, and results were visualized using scatter plots. Figure 1 illustrates the relationship between trip_distance and fare_amount, with a regression line demonstrating a strong positive correlation. Figure 2 shows the predicted vs. actual fare amounts plot, where most predictions cluster along the diagonal, indicating strong alignment with actual values. These results highlight the model's accuracy and reliability in predicting taxi fares based on trip distance.

## Results

The analysis aimed to predict fare amounts using a linear regression model. The regression plot confirmed a positive linear relationship between trip_distance and fare_amount, with fares increasing by approximately 3.62 USD per mile and a base fare of around 7.02 USD. The model demonstrated moderate predictive power with an R-squared value of 0.848, explaining 84.8% of the variance, a low MAE of 3.24 USD indicating accuracy for typical rides, and an RMSE of 6.78 USD reflecting sensitivity to outliers. While the model performed well in the 10–50 USD range, it underestimated fares over 100 USD due to factors like peak hour fees and tolls, which were not included in the model. The test data evaluation and scatter plots revealed the model's general fit but also highlighted some outliers, suggesting areas for improvement with more complex models or additional features.

# Limitations and Next Steps

Overall, our model may be useful in the initial analysis of tourists interested in making informal predictions about NYC taxi fare amounts. However, there are several areas where this work can be improved. In this analysis, we did not include hyperparameters like the regularization parameter (α), which is essential for controlling overfitting and improving model generalizability. Furthermore, the model currently uses only a single feature (trip distance), which may result in underfitting and bias. In the next steps, we plan to incorporate more features into the model and refine the approach to improve its precision and predictive power. Additionally, experimenting with other regression models, such as KNN regression for non-linear relationships and Lasso regression for linear relationships, could be valuable to see if these models improve performance.

# References

Charles R Harris, K Jarrod Millman, Stéfan J van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J Smith, Robert Kern, Matti Picus, Stephan Hoyer, Marten H van Kerkwijk, Matthew Brett, Allan Haldane, Jaime Fernández del Río, Mark Wiebe, Pearu Peterson, Pierre Gérard-Marchant, Kevin Sheppard, Tyler Reddy, Warren Weckesser, Hameer Abbasi, Christoph Gohlke, and Travis E Oliphant. Array programming with NumPy. Nature, 585(7825):357–362, 2020. URL: https://doi.org/10.1038/s41586-020-2649-2, doi:10.1038/s41586-020-2649-2.

F. Pedregosa, G. Varoquaux, A. Gramfort, V. Michel, B. Thirion, O. Grisel, M. Blondel, P. Prettenhofer, R. Weiss, V. Dubourg, J. Vanderplas, A. Passos, D. Cournapeau, M. Brucher, M. Perrot, and E. Duchesnay. Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research, 12:2825–2830, 2011.

Jake VanderPlas. Altair: interactive statistical visualizations for python. Journal of open source software, 3(7825):1057, 2018. URL: https://doi.org/10.21105/joss.01057, doi:10.21105/joss.01057.

Guido Van Rossum and Fred L. Drake. Python 3 Reference Manual. CreateSpace, Scotts Valley, CA, 2009. ISBN 1441412697.

New York City Taxi and Limousine Commission. TLC Trip Record Data. Retrieved from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page, 2024.

Wes McKinney. Data structures for statistical computing in python. In Stéfan van der Walt and Jarrod Millman, editors, Proceedings of the 9th Python in Science Conference, =51 – 56. 2010.






In [ ]:
# !jupyter nbconvert --to webpdf yellow_taxi_analysis.ipynb